#### 使用Scipy求解SVM

In [1]:
from scipy.optimize import minimize
import numpy as np
class SVM:
    def __init__(self,C=0.001,tol=1e-4):
        self.c=C
        self.relaxation_type='eq' if C==0 else 'ineq'
        self.tol=tol
        
    def fit(self,X,y):
        y=2*y-1
        m,n=X.shape
        unknow_vector=np.ones(m+n+1)/(m+n+1)
        
        self.solver=minimize(self.minimize_object,
                             unknow_vector,args=(X,y,self.c),
              constraints=({'type': 'ineq', 'args': (X,y),
                            'fun':lambda unknow_vector,X,
                            y:self.linear_onstraint(unknow_vector,X,y)},
                           {'type':self.relaxation_type, 
                            'args': (y,),'fun':lambda unknow_vector,
                            y:self.relaxation_constraint(unknow_vector,y)}                         
             ))
        self.coef_=self.solver.x[:n]
        self.intercept_=b=self.solver.x[n]
        self.beta_=self.solver.x[n+1:]
        
        self.support_value=y*(np.dot(X,self.coef_.T)+self.intercept_)+self.beta_
        self.support_=np.where(self.support_value<=1+self.tol)
        self.support_vectors_=X[self.support_]
           
        return self
    
    def predict(self,X):
        pred=X@self.coef_+self.intercept_
        return (np.sign(pred)+1)/2
        
    
    
    @staticmethod
    def minimize_object(unknow_vector,X,y,c):
        m,n=X.shape
        w=unknow_vector[:n]  
        b=unknow_vector[n]   
        beta=unknow_vector[n+1:]
        object_value=0.5*np.sum(w**2)+c*np.sum(beta)
        return object_value
    
    @staticmethod
    def linear_onstraint(unknow_vector,X,y):
        m,n=X.shape
        w=unknow_vector[:n]
        b=unknow_vector[n]
        beta=unknow_vector[n+1:]
        return y*(X@w.T+b)+beta-1
    
    @staticmethod
    def relaxation_constraint(unknow_vector,y):
        n=len(y)
        beta=unknow_vector[n+1:]    
        return beta

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X,y=load_breast_cancer().data,load_breast_cancer().target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
my_svm=SVM(C=1).fit(X_train,y_train)
pred=my_svm.predict(X_test)
pred

array([0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
       1.])

In [3]:
accuracy_score(y_test,pred)

0.9298245614035088

In [4]:
my_svm.support_

(array([ 9, 46, 63]),)

In [5]:
my_svm.support_value[my_svm.support_value<1.0001]

array([-0.21180606,  0.94134681,  0.39603054])